In [0]:
from pyspark.sql.functions import col, avg, count, when

spark.sql("CREATE DATABASE IF NOT EXISTS hotel_gold")
df = spark.table("hotel_silver.bookings_clean")

kpis = df.groupBy("hotel").agg(
    avg("adr").alias("avg_adr"),
    avg("revenue").alias("avg_revenue"),
    count("*").alias("total_bookings"),
    avg(when(col("is_canceled") == 1, 1).otherwise(0)).alias("cancellation_rate")
)

kpis.write.format("delta") \
  .mode("overwrite") \
  .saveAsTable("hotel_gold.daily_kpis")
